In [1]:
# set the working directory
import os as os
os.chdir("C:/Users/User/Desktop/18-2/DM/data")
print(os.getcwd())

def read():
    import pandas as pd
    name = input("Enter the data file name (with extension name) : ")
    fm = input("Select the data coding format(1='a b c' or 2='a,b,c'): " )
    if fm == '1':
        form = " "
    elif fm == '2':
        form = ","
    return pd.read_csv(name, sep=form, header=None)

C:\Users\User\Desktop\18-2\DM\data


In [5]:
def classification():
    # import packages
    import math
    import pandas as pd
    import numpy as np
    import numpy.linalg as lin
    from fractions import Fraction as frac

    # prompt user to enter the data
    print('Importing TRAINING data set')
    train = read()
    print('Importing TRAINING data set')
    test = read()
    num = int(input("Enter which column the response variable is recorded: "))-1


IndentationError: unexpected indent (<ipython-input-5-f487b31585b1>, line 3)

In [711]:
import math
import pandas as pd
import numpy as np
import numpy.linalg as lin
from fractions import Fraction as frac

In [3]:
print('Importing TRAINING data set')
train = read()

Importing TRAINING data set
Enter the data file name (with extension name) : veh.dat
Select the data coding format(1='a b c' or 2='a,b,c'): 2


In [4]:
print('Importing TRAINING data set')
test = read()

Importing TRAINING data set
Enter the data file name (with extension name) : vehtest.dat
Select the data coding format(1='a b c' or 2='a,b,c'): 2


In [5]:
num = int(input("Enter which column the response variable is recorded: "))-1

Enter which column the response variable is recorded: 19


In [7]:
choice = input("Enter 1 to implement LDA or enter 2 to implement QDA")

Enter 1 to implement LDA or enter 2 to implement QDA2


In [8]:
nclass = len(train[num].unique())
# Assume that values of the class variable are integers starting with 1

In [109]:
print(' It has',nclass,'Classes.','\n', 'Input each priors with ascending order of Class index.','\n',
      'ex) Priors of 3 class size = 1/3, 1/3, 1/3','\n')
prior = input('Priors : ').split(',')

 It has 4 Classes. 
 Input each priors with ascending order of Class index. 
 ex) Priors of 3 class size = 1/3, 1/3, 1/3 

Priors : 1/5,1/5,1/5,1/5


In [110]:
prior = list(map(lambda x: frac(x), prior))

In [111]:
if not len(prior) == nclass :
    prior = [1/nclass]*nclass
    print("Your prior input does not correscond with class size.")
    print("So equal prior is given such as", prior)
elif not sum(prior)==1 :
    prior = [1/nclass]*nclass
    print("Sum of your prior input is not equal to 1.")
    print("So equal prior is given such as", prior)

Sum of your prior input is not equal to 1.
So equal prior is given such as [0.25, 0.25, 0.25, 0.25]


In [712]:
# Basic stat
N = train.shape[0]
n = test.shape[0]
np = train[num].groupby(train[num]).count()
x = train.drop(num, axis=1).T
x_test = test.drop(num, axis=1).T
y = train[num]
y_test = test[num]
prior = pd.DataFrame(prior)

In [713]:
means = train.groupby(train[num]).mean().T
cov = train.groupby(train[num]).cov()

In [714]:
covs=[]
sp = 0
for i in range(1, nclass+1):
    covs.append(cov.loc[i])
    sp = sp + (np[i]-1)*cov.loc[i]/(N-nclass)

In [715]:
# LDA
choice=2
if choice == 1 :
    dresub=[]
    dtest=[]

    for i in range(1, nclass+1):
        t0 = (means[i].T).dot(lin.inv(sp)).dot(x)
        t1 = (means[i].T).dot(lin.inv(sp)).dot(x_test)
        t2 = (-0.5)*means[i].T.dot(lin.inv(sp)).dot(means[i])+math.log(prior.T[i-1])
        t3 = (-0.5)*means[i].T.dot(lin.inv(sp)).dot(means[i])+math.log(prior.T[i-1])
        dresub.append(t0+t2)
        dtest.append(t1+t3)
    y = pd.concat([pd.DataFrame(y),pd.DataFrame(dresub).idxmax()+1], axis=1)
    y_test = pd.concat([pd.DataFrame(y_test),pd.DataFrame(dtest).idxmax()+1], axis=1)
    
    
elif choice == 2 :
    import numpy as np
    dresub = np.zeros((nclass,N))
    dtest = np.zeros((nclass,n))

    for i in range(0, nclass):
        for j in range(0, N):
            t0 = (-0.5)*math.log(lin.det(cov.loc[i+1]))
            t1 = (-0.5)*((x[j]-means[i+1]).T).dot(lin.inv(cov.loc[i+1])).dot(x[j]-means[i+1]) + math.log(prior.T[i])
            dresub[i,j]= t0+t1
        for j in range(0, n):
            t2 = (-0.5)*((x_test[j]-means[i+1]).T).dot(lin.inv(cov.loc[i+1])).dot(x_test[j]-means[i+1]) + math.log(prior.T[i])
            dtest[i,j]= t0+t2

    y_pred = pd.DataFrame(dresub).idxmax()+1
    y_test_pred = pd.DataFrame(dtest).idxmax()+1

In [726]:
# Crosstable
cross_res = pd.crosstab(y, y_pred,rownames=['Actual Class'], colnames=['Predicted Class'])
accuracy_res = np.trace(cross_res)/N

cross_t = pd.crosstab(y_test, y_test_pred,rownames=['Actual Class'], colnames=['Predicted Class'])
accuracy_t = np.trace(cross_t)/N

In [728]:
# output file name
outputname = input("Write the output file name you want to save (without extension name) : ")
outputname = outputname+'.txt'

Write the output file name you want to save (without extension name) : hw3tttteessssttttttttt


In [731]:
# outport the result
with open(outputname,"w") as text_file:
    
    print('ID, Actual class, Resub pred', file=text_file)
    print('-----------------------------', file=text_file)
    for i in range(N):
         print(i+1, y[i], y_pred[i], sep=', ', file=text_file)
    print('',file=text_file)
    print('Confusion Matrix (Resubstitution)', file=text_file)
    print('----------------------------------', file=text_file)
    print(cross_res, file=text_file)
    print("",file=text_file)
    print("Model Summary (Resubstitution)", file=text_file)
    print('------------------------------', file=text_file)
    print("Overall accuracy = ", accuracy_res.round(3), sep='', file=text_file)
    print('', file=text_file)
    
    print('ID, Actual class, Test pred', file=text_file)
    print('-----------------------------', file=text_file)
    for i in range(n):
        print(i+1, y[i], y_pred[i], sep=', ', file=text_file)
    print('',file=text_file)
    print('Confusion Matrix (Test)', file=text_file)
    print('----------------------------------', file=text_file)
    print(cross_t, file=text_file)
    print("",file=text_file)
    print("Model Summary (Test)", file=text_file)
    print('------------------------------', file=text_file)
    print("Overall accuracy = ", accuracy_t.round(3), sep='', file=text_file)
    print('', file=text_file)

In [730]:
n

336